In [ ]:
!pip install xlearn

     |████████████████████████████████| 4.9 MB 13.0 MB/s 
  Created wheel for xlearn: filename=xlearn-0.40a1-py3-none-any.whl size=229990 sha256=18a8d1bc00b1f517b8b8da0f1456bd402164e108b05e25aeaa4e8155ecb6299a
  Stored in directory: /root/.cache/pip/wheels/1e/6d/eb/256faaf8142a9363431e2dc3f6a3d1c9080f526a1e5475049d
Successfully built xlearn


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import xlearn as xl
import numpy as np
import pandas as pd
import math
import gc
import os

from sklearn.model_selection import train_test_split

In [ ]:
valid_data = pd.read_csv('/content/drive/MyDrive/Taobao/valid_data_10_percent.csv')

In [ ]:
valid_data.columns

Index(['index', 'userid', 'time_stamp', 'adgroup_id', 'pid', 'nonclk', 'clk',
       'cate_id', 'campaign_id', 'customer', 'brand', 'price', 'cms_segid',
       'cms_group_id', 'final_gender_code', 'age_level', 'pvalue_level',
       'shopping_level', 'occupation', 'new_user_class_level', 'time', 'day',
       'hour', 'day_hour', 'weekday', 'num_show_last_hour_user_adgroup',
       'num_click_last_hour_user_adgroup', 'num_show_last_day_user_adgroup',
       'num_click_last_day_user_adgroup', 'num_show_last_hour_user_cate',
       'num_click_last_hour_user_cate', 'num_show_last_day_user_cate',
       'num_click_last_day_user_cate', 'num_show_last_hour_user_customer',
       'num_click_last_hour_user_customer', 'num_show_last_day_user_customer',
       'num_click_last_day_user_customer', 'num_show_last_hour_user_brand',
       'num_click_last_hour_user_brand', 'num_show_last_day_user_brand',
       'num_click_last_day_user_brand', 'num_show_last_hour_user_all',
       'num_click_last_hou

In [ ]:
num_train = len(valid_data[valid_data['userid']<=valid_data.loc[int(0.8*len(valid_data))]['userid']])
num_train

1758557

In [ ]:
# numerics = ['price']  

numerics = ['price', 
      #   'num_show_last_hour_user_adgroup', 'num_click_last_hour_user_adgroup',
      #  'num_show_last_day_user_adgroup', 'num_click_last_day_user_adgroup',
      #  'num_show_last_hour_user_cate', 'num_click_last_hour_user_cate',
      #  'num_show_last_day_user_cate', 'num_click_last_day_user_cate',
      #  'num_show_last_hour_user_customer', 'num_click_last_hour_user_customer',
      #  'num_show_last_day_user_customer', 'num_click_last_day_user_customer',
      #  'num_show_last_hour_user_brand', 'num_click_last_hour_user_brand',
      #  'num_show_last_day_user_brand', 'num_click_last_day_user_brand',
      #  'num_show_last_hour_user_all', 'num_click_last_hour_user_all',
      #  'num_show_last_day_user_all', 'num_click_last_day_user_all',
      #  'num_show_last_hour_all_adgroup', 'num_click_last_hour_all_adgroup',
      #  'num_show_last_hour_all_cate', 'num_click_last_hour_all_cate',
      #  'num_show_last_hour_all_customer', 'num_click_last_hour_all_customer',
      #  'num_show_last_hour_all_brand', 'num_click_last_hour_all_brand'
       ]  

categories = ['cate_id','customer', 'brand', 'cms_segid', 'cms_group_id', 'final_gender_code', 'age_level',
       'pvalue_level', 'shopping_level', 'occupation', 'new_user_class_level','hour','weekday']
       
valid_cols = numerics + categories + ['clk']

In [ ]:
for col in numerics:
  
  valid_data[col].fillna(0, inplace=True)
  # valid_data[col] = pd.qcut(valid_data[col], 20, labels=False, duplicates = "drop")

In [ ]:
import datetime
def df2libffm(df, num_train, save_file=None, y=None, non_categorical_cols = []) :
    
    assert y is not None
 
    row_cnt = df.shape[0]
 
    out = pd.DataFrame({
        y: df[y]
    })
 
    print("out.shape:", out.shape )
    df = df.drop(columns=[y], axis=1)
    
    feature_base = 0
 
    for idx, col in enumerate(df.columns.tolist()) :
        dt = datetime.datetime.now()

        print(str(dt), idx, col)
 
        cur_field_id = idx
        field_series = pd.Series([cur_field_id] * row_cnt).astype(str)
 
        if col in non_categorical_cols :
            # if not categorical feature, do not consider how many different values
            feature_series = pd.Series([feature_base] * row_cnt).astype(str)
            # print(len(feature_series))
            feature_base += 1
            value_series = df[col].astype(str)
            # print(len(value_series))
 
            new_col = field_series + ":" + feature_series + ":" + value_series
            out[str(cur_field_id)] = new_col.values
            
        else :
            # if is categorical feature
            feature_series = df[col].astype('category').values.codes.astype('int32')
            feature_series = feature_series + feature_base
            feature_series = pd.Series(feature_series).astype(str)
 
            feature_base += feature_series.unique().shape[0]
            print("next feature base:", feature_base)
 
            new_col = field_series + ":" + feature_series + ":1" 
            out[str(cur_field_id)] = new_col.values
 
    if save_file:
        file_name = save_file
        print("save file name:", file_name)
        
        out.loc[:int(num_train)].to_csv(file_name + "_train_validate.txt", sep=" ", header=False, index=False)
        
        # 随机划分
        
        temp_train, temp_val = train_test_split(out.loc[:int(num_train)], test_size=0.2, random_state=123)
        
        temp_train.to_csv(file_name + "_train.txt", sep=" ", header=False, index=False)
        temp_val.to_csv(file_name + "_validate.txt", sep=" ", header=False, index=False)
        
        out.loc[int(num_train):].to_csv(file_name + "_test.txt", sep=" ", header=False, index=False)
 
    return out

In [ ]:
df2libffm(valid_data[valid_cols], num_train, "fm", y = 'clk', non_categorical_cols = numerics)

out.shape: (2198169, 1)
2022-04-09 07:43:03.062781 0 price
2022-04-09 07:43:08.534549 1 cate_id
next feature base: 5288
2022-04-09 07:43:12.583208 2 customer
next feature base: 152539
2022-04-09 07:43:16.755528 3 brand
next feature base: 216358
2022-04-09 07:43:23.515442 4 cms_segid
next feature base: 216455
2022-04-09 07:43:27.829676 5 cms_group_id
next feature base: 216468
2022-04-09 07:43:31.768391 6 final_gender_code
next feature base: 216470
2022-04-09 07:43:35.650930 7 age_level
next feature base: 216477
2022-04-09 07:43:39.569716 8 pvalue_level
next feature base: 216481
2022-04-09 07:43:43.468271 9 shopping_level
next feature base: 216484
2022-04-09 07:43:47.369095 10 occupation
next feature base: 216486
2022-04-09 07:43:51.340346 11 new_user_class_level
next feature base: 216491
2022-04-09 07:43:55.271157 12 hour
next feature base: 216515
2022-04-09 07:43:59.261294 13 weekday
next feature base: 216522
save file name: fm


,clk,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0,0:0:129.0,1:1097:1,2:12270:1,3:157782:1,4:216358:1,5:216458:1,6:216469:1,7:216473:1,8:216476:1,9:216482:1,10:216484:1,11:216485:1,12:216511:1,13:216521:1
1,0,0:0:12.9,1:2243:1,2:149242:1,3:152538:1,4:216358:1,5:216458:1,6:216469:1,7:216473:1,8:216476:1,9:216482:1,10:216484:1,11:216485:1,12:216506:1,13:216519:1
2,0,0:0:108.0,1:495:1,2:148240:1,3:194880:1,4:216358:1,5:216458:1,6:216469:1,7:216473:1,8:216476:1,9:216482:1,10:216484:1,11:216485:1,12:216498:1,13:216518:1
3,0,0:0:69.0,1:2243:1,2:131845:1,3:173080:1,4:216358:1,5:216458:1,6:216469:1,7:216473:1,8:216476:1,9:216482:1,10:216484:1,11:216485:1,12:216512:1,13:216521:1
4,0,0:0:9.9,1:1894:1,2:6393:1,3:205082:1,4:216358:1,5:216458:1,6:216469:1,7:216473:1,8:216476:1,9:216482:1,10:216484:1,11:216485:1,12:216502:1,13:216518:1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2198164,0,0:0:139.0,1:1015:1,2:48835:1,3:152538:1,4:216360:1,5:216456:1,6:216469:1,7:216471:1,8:216478:1,9:216483:1,10:216484:1,11:216488:1,12:216503:1,13:216517:1
2198165,0,0:0:74.99,1:1015:1,2:6654:1,3:152538:1,4:216360:1,5:216456:1,6:216469:1,7:216471:1,8:216478:1,9:216483:1,10:216484:1,11:216488:1,12:216501:1,13:216517:1
2198166,0,0:0:225.0,1:879:1,2:90424:1,3:152538:1,4:216358:1,5:216464:1,6:216468:1,7:216473:1,8:216476:1,9:216481:1,10:216484:1,11:216485:1,12:216511:1,13:216517:1
2198167,0,0:0:139.0,1:2030:1,2:86592:1,3:160877:1,4:216358:1,5:216464:1,6:216468:1,7:216473:1,8:216476:1,9:216481:1,10:216484:1,11:216485:1,12:216511:1,13:216517:1


In [ ]:
# parameter 1: ffm/fm/lr
# parameter 2: learning rate
# parameter 3: lambda（penalty）
# parameter 4: k
# parameter 5: metric —— 'acc', 'prec', 'recall', 'f1', 'auc'
# parameter 6: epoch

In [ ]:
!python /content/drive/MyDrive/Taobao/FFM.py "fm" 0.1 0.00000001 12 'auc' 200 '0320'

----------------------------------------------------------------------------------------------
           _
          | |
     __  _| |     ___  __ _ _ __ _ __
     \ \/ / |    / _ \/ _` | '__| '_ \ 
      >  <| |___|  __/ (_| | |  | | | |
     /_/\_\_____/\___|\__,_|_|  |_| |_|

        xLearn   -- 0.40 Version --
----------------------------------------------------------------------------------------------

[------------] xLearn uses 4 threads for training task.
[ ACTION     ] Read Problem ...
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (fm_train.txt.bin) found. Skip converting text to binary.
[------------] First check if the text file has been already converted to binary format.
[------------] Binary file (fm_validate.txt.bin) found. Skip converting text to binary.
[------------] Number of Feature: 216522
[------------] Time cost for reading problem: 1.31 (sec)
[ ACTION     ] Initialize model ...
[------------]

In [ ]:
pd.read_csv("output_test.txt", header = None).to_csv("/content/drive/MyDrive/Taobao/FM_before.csv", index = False, header = False)